<b><font size="6" color="red">ch10 데이터베이스연동</font></b>

# 1절 SQLite 데이터 베이스연결
- SQLite 데이터 베이스는 별도의 DBMS 없이 SQL을 이용하여 DB 엑세스할 수 있도록 만든 **간단한 디스크 기반의 데이터베이스를 제공**하는 C라이브러리
- SQLite는 프로토타입(시제품 나오기전까지 제품)을 만들때 사용. 정식 제품은 오라클이나 다른 DBMS를 이식
- [DB browser for SQLite](http://sqlitebrowser.org/dl)

## 1.1 SQLite browser 설치 및 sqlite3 패키지 load

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.35.4'

In [2]:
import pandas as pd
pd.__version__

'1.2.4'

## 1.2 데이터 베이스 연결

In [3]:
# DB 연결 생성시, 파일이 있으면 그 파일 연결. 파일이 없으면 빈 파일을 생성
conn = sqlite3.connect('data/ch10_example.db')
conn

In [4]:
# 커서 객체 반환. 커서는 SQL문 실행시키고 결과데이터(SELECT)를 조회하는데 사용객체
cursor = conn.cursor()
cursor

In [5]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

In [7]:
cursor.execute("DROP TABLE MEMBER") #2번실행 테이블 없으면 오류

OperationalError: no such table: MEMBER

In [8]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

In [11]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동',20,'h@h.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('김길동',20,'h@h.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('박길동',20,'h@h.com')")

In [12]:
conn.commit() #반대 : conn.rollback() DML 문에서만 commit

In [13]:
# select 실행결과는 cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER ORDER BY NAME")

In [14]:
# select 문의 결과를 받는 함수
## fetchall() - 결과를 모두 받을 때(튜플 list)
## fetchone() - 결과를 한행씩 받을 때(튜플)
## fetchmany(n) - 결과를 n행 받을 때
print(cursor.fetchall())

[('홍길동', 20, 'h@h.com'), ('김길동', 20, 'h@h.com'), ('박길동', 20, 'h@h.com')]


In [15]:
print(cursor.fetchall())  #한번 소요된 커서는 다시 패치할 수 없다.

[]


In [16]:
cursor.execute("SELECT * FROM MEMBER")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'), ('김길동', 20, 'h@h.com'), ('박길동', 20, 'h@h.com')]

In [17]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('김길동', 20, 'h@h.com')
('박길동', 20, 'h@h.com')


In [19]:
cursor.execute("SELECT * FROM MEMBER")
member_list = []
while True:
    member = cursor.fetchone()
    if member is None:
        break
    member_list.append({'name':member[0], 'age':member[1], 'email':member[2]})
    print('이름 :',member[0], '\t나이 :',member[1], '\t메일 :', member[2])

이름 : 홍길동 	나이 : 20 	메일 : h@h.com
이름 : 김길동 	나이 : 20 	메일 : h@h.com
이름 : 박길동 	나이 : 20 	메일 : h@h.com


In [20]:
member_list

[{'name': '홍길동', 'age': 20, 'email': 'h@h.com'},
 {'name': '김길동', 'age': 20, 'email': 'h@h.com'},
 {'name': '박길동', 'age': 20, 'email': 'h@h.com'}]

In [23]:
cursor.execute('SELECT * FROM MEMBER ORDER BY AGE DESC')
for member in cursor.fetchmany(2):
    print(member[0], member[1], member[2])

홍길동 20 h@h.com
김길동 20 h@h.com


In [24]:
cursor.execute('SELECT * FROM MEMBER ORDER BY AGE DESC')
for member in cursor.fetchmany(10):
    print(member[0], member[1], member[2])

홍길동 20 h@h.com
김길동 20 h@h.com
박길동 20 h@h.com


In [25]:
cursor.close() # cursor.close는 생략가능
conn.close()

## 1.3 SQL 구문에 파라미터 사용하기
- qmark, named(추천) 방법

In [26]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','유낄똥')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com')]

In [29]:
# 파라미터 사용하기 : qmark 방법 이용
name1 = input('검색할 이름1은 ?')
name2 = input('검색할 이름2는 ?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?, ?)",(name1,name2))
cursor.fetchall()

검색할 이름1은 ?홍길동
검색할 이름2는 ?유낄동


[('홍길동', 20, 'h@h.com')]

In [30]:
name1 = input('검색할 이름1은 ?')
name2 = input('검색할 이름2는 ?')
names = (name1, name2) #튜플이나 리스트형 변수로 만들어 저야 한다.(순서가 있어야 한다.)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)",names)
cursor.fetchall()

검색할 이름1은 ?홍길동
검색할 이름2는 ?김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 20, 'h@h.com')]

In [32]:
# 파라미터 사용하기 : named 방법 이용
name = input('검색할 이름1은 ?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME = :name ",{'name':name1})
members = cursor.fetchall()
if len(members) == 0:
    print("해당 이름의 데이터가 없습니다.")
else:
    print(members)



검색할 이름1은 ?홍길동
[('홍길동', 20, 'h@h.com')]


In [ ]:
# member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 입력받아 DB insert하는 프로그램 수행)

In [ ]:
# 파라미터 사용하기 : named 방법 이용
name1 = input('검색할 이름은 ?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME =  :name ", {'name':name1})
members = cursor.fetchall()
if len(members) == 0:
    print("해당 이름의 데이터가 없습니다")
else:
    print(members)
    # print(cursor.fetchall()) # 무조건 한행도 출력 안 됨

In [ ]:
# member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 입력받아 DB에 insert하는 프로그램 수행)
try:
    name = input('입력할 이름은 ?')
    age  = int(input('입력할 나이는 ?'))
    email = input('입력할 메일은 ? ')
except:
    print('유효하지 않는 나이 입력시 18세로 초기화')
    age = 18
    email = input('입력할 메일은 ?')
inputdata = {'name':name, 'age':age, 'email':email}
# cursor.execute("INSERT INTO MEMBER VALUES (?, ?, ?)", (name, age, email))
# cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", inputdate)
cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", 
                      {'name':name, 'age':age, 'email':email})
conn.commit()

In [34]:
#입력
a = int(input("몇개의 데이터를 입력하시겠습니까?"))
for i in range(a):
    name_i = input("이름 입력")
    age_i = int(input("나이 입력"))
    email_i = input("이메일 입력")
    
    cursor.execute("INSERT INTO MEMBER VALUES (?,?,?)",(name_i,age_i,email_i))

conn.commit()

몇개의 데이터를 입력하시겠습니까?1
이름 입력포길동
나이 입력5
이메일 입력po@po.com


In [ ]:
# member 테이블에 입력0입력 전까지 (사용자로부터 이름, 나이, 메일을 입력받아 DB insert하는 프로그램 수행)
while True:
    a=int(input("입력여부 선택 0=입력취소"))
    if a==0:
        break
    name_i = input("이름 입력")
    age_i = int(input("나이 입력"))
    email_i = input("이메일 입력")
    
    cursor.execute("INSERT INTO MEMBER VALUES (?,?,?)",(name_i,age_i,email_i))
    
conn.commit()

In [38]:
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

[('홍길동', 20, 'h@h.com'),
 ('김길동', 20, 'h@h.com'),
 ('박길동', 20, 'h@h.com'),
 ('포길동', 5, 'po@po.com'),
 ('키길동', 3, 'ki@ki.com')]

In [35]:

class User:
    '이름, 나이, 메일이 있는 class'
    def __init__(self, name, age, email):
        self.name = name
        self.age  = age
        self.email = email
    def __str__(self):
        return '이름:{:4}\t나이:{:3}\t메일:{}'.format(self.name, self.age, self.email)
    def as_dict(self):
        return {'name':self.name, 'age':self.age, 'email':self.email}

try:
    name = input('입력할 이름은 ?')
    age  = int(input('입력할 나이는 ?'))
    email = input('입력할 메일은 ? ')
except:
    print('유효하지 않는 나이 입력시 18세로 초기화')
    age = 18
    email = input('입력할 메일은 ?')
user = User(name, age, email)
cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", user.as_dict())
conn.commit()

In [37]:
# member 테이블에 입력(0을 입력할 때까지 계속 입력)
user_list = [] # 입력한 user 객체들이 저장될 예정
while True:
    try:
        name = input('입력할 이름은 ?(입력을 종료하고자 하면 0입력)')
        if name == '0':
            break
        age  = int(input('입력할 나이는 ?'))
        email = input('입력할 메일은 ? ')
    except:
        print('유효하지 않는 나이 입력시 18세로 초기화')
        age = 18
        email = input('입력할 메일은 ?')
    user = User(name, age, email)
    user_list.append(user)
    cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", user.as_dict())
conn.commit()
print('입력된 데이터는 ', len(user_list),'명 추가되었습니다.')
for user in user_list:
    print(user)

입력할 이름은 ?(입력을 종료하고자 하면 0입력)키길동
입력할 나이는 ?3
입력할 메일은 ? ki@ki.com
입력할 이름은 ?(입력을 종료하고자 하면 0입력)0
입력된 데이터는  1 명 추가되었습니다.
이름:키길동 	나이:  3	메일"ki@ki.com


In [39]:
cursor.close() # cursor.close는 생략가능
conn.close()   

# 2절. 오라클 데이터 베이스 연결

In [43]:
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host="localhost",port=1521,sid='xe')
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [44]:
# (1) conn을 얻어오는 방법1
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
conn
#여기서 에러날 시 VC_redist.x64.exe 인스톨

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [45]:
conn.close()

In [47]:
# (2) conn을 얻어오는 방법2
conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [48]:
# cursor 얻어오기
cursor = conn.cursor()

In [49]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [50]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [51]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
for e in cursor:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [52]:
sql = "SELECT * FROM EMP WHERE ENAME =:ename"
ename = input("검색할 이름은 ?")
cursor.execute(sql, {'ename':ename})
emp = cursor.fetchall()
if len(emp) == 0:
    print('입력하신 이름의 데이터는 없습니다.')
else:
    for e in emp:
        print(e)

검색할 이름은 ?SMITH
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)


In [53]:
# 데이터 베이스 검색 결과를 DataFrame으로 
import pandas as pd
cursor.execute('SELECT * FROM EMP')
data = cursor.fetchall() #튜플 리스트
data_df = pd.DataFrame(data) #데이터 프레임
data_df


,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17 00:00:00,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20 00:00:00,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22 00:00:00,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02 00:00:00,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28 00:00:00,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01 00:00:00,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09 00:00:00,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09 00:00:00,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17 00:00:00,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08 00:00:00,1500.0,0.0,30


In [54]:
type(data_df)

pandas.core.frame.DataFrame

In [55]:
# 각 필드의 정보 (필드명, type, display_size, 정확도, scale, nullable....)
cursor.description #튜플리스트로 출력

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [59]:
[ row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [60]:
data_df.columns

RangeIndex(start=0, stop=8, step=1)

In [61]:
data_df.columns = [ row[0] for row in cursor.description] #열이름 변경
data_df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17 00:00:00,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20 00:00:00,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22 00:00:00,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02 00:00:00,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28 00:00:00,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01 00:00:00,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09 00:00:00,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09 00:00:00,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17 00:00:00,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08 00:00:00,1500.0,0.0,30


In [62]:
data_df.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [63]:
cursor.close()
conn.close()

# 3절 연습문제

In [64]:
# 클래스 생성은 선택사항

In [65]:
# 1. 입력
def insert_member():
    # 사용자로 부터 이름, 전화번호, 이메일, 나이, 등급(1-5), 기타특징 입력받아 DB에 추가한다.
    pass

In [66]:
#위의 메소드 test

In [67]:
# 2. 전체 출력
def select_all():
    #"SELECT * FROM MEMBER"의 결과를 출력(DataFrame형태 / 리스트형태)
    pass
    

In [68]:
# 위의 메소드 test

In [69]:
# 3. 이름 검색
def select_name():
    #사용자로부터 검색하고자 하는 이름을 받아 결과 출력(DataFrame형태 / 리스트형태)
    pass

In [70]:
# 위의 메소드 test

In [71]:
#4. 메일 삭제
def delete_email():
    # 사용자로부터 삭제할 메일을 입력받아 해당 메일이 있는지 확인하고
    # 해당 이메일이 없으면 없다 출력. 있으면 삭제하고 삭제했다 출력
    pass

In [72]:
#위의 메소드 test

In [ ]:
#지금 현재 DB 내용
cursor.execute("select * from member")
cursor.fetchall()

In [73]:
# 5. CSV 내보내기
def save_csv():
    # 'SELECT * FROM MEMBER의 결과를 CSV로 내보내기(HEADER 포함)'
    pass

In [74]:
#위의 메소드 test

In [ ]:
#0. 종료
def close_sql():
    global conn
    conn.close()
    print("종료합니다.")